In [1]:
from IPython.core.display import display, HTML 
display(HTML("<style>.container { width:100% !important; }</style>"))
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
%matplotlib inline

In [2]:
import seaborn as sns
import numpy as np

df = sns.load_dataset("iris")

In [3]:
def max_devided_min(x):
    return max(x) / min(x)

def max_minus_min(x):
    return max(x) - min(x)

aggs = {'nunique', 'count', 'sum', 'min', 'mean', 'median', 'max', 'var', 'quantile',  max_minus_min, max_devided_min}

In [4]:
# X = df.drop(['sepal_length'], axis=1)
X = df
y = df['sepal_length']

In [5]:
from sklearn.model_selection import train_test_split
(X_train, X_test,
 y_train, y_test) = train_test_split(
    X, y, test_size=0.2, random_state=0,
)

In [6]:
X_train.head()

,sepal_length,sepal_width,petal_length,petal_width,species
137,6.4,3.1,5.5,1.8,virginica
84,5.4,3.0,4.5,1.5,versicolor
27,5.2,3.5,1.5,0.2,setosa
127,6.1,3.0,4.9,1.8,virginica
132,6.4,2.8,5.6,2.2,virginica


In [7]:
len(X_train)

120

In [8]:
from sklearn.model_selection import KFold

def target_encoding(X_train, key_col):
    kf = KFold(n_splits=5, shuffle=True, random_state=0)
    dfs = []

    for train_index, test_index in kf.split(X_train):
        X_used, X_unused = X_train.loc[train_index, :], X_train.loc[test_index, :]

        agg = X_used.groupby(key_col).agg(aggs)
        names_1 = agg.columns.get_level_values(0)
        names_2 = agg.columns.get_level_values(1)
        agg.columns = ['agg_' + el1 + '_' + el2 for (el1, el2) in zip(names_1, names_2)]
        tmp = pd.merge(X_unused, agg, on=key_col, how='left')
        tmp.index = test_index
        dfs.append(tmp)

    return pd.concat(dfs).sort_index()

In [9]:
X_train = target_encoding(X_train, 'species')

In [10]:
X_train.head()

,sepal_length,sepal_width,petal_length,petal_width,species,agg_sepal_length_var,agg_sepal_length_sum,agg_sepal_length_max_minus_min,agg_sepal_length_min,agg_sepal_length_mean,...,agg_petal_width_sum,agg_petal_width_max_minus_min,agg_petal_width_min,agg_petal_width_mean,agg_petal_width_quantile,agg_petal_width_max,agg_petal_width_max_devided_min,agg_petal_width_median,agg_petal_width_count,agg_petal_width_nunique
0,5.1,3.5,1.4,0.2,setosa,0.121577,185.7,1.5,4.3,5.018919,...,9.0,0.5,0.1,0.243243,0.2,0.6,6.0,0.2,37,6
1,4.9,3.0,1.4,0.2,setosa,0.121577,185.7,1.5,4.3,5.018919,...,9.0,0.5,0.1,0.243243,0.2,0.6,6.0,0.2,37,6
2,4.7,3.2,1.3,0.2,setosa,0.126771,190.6,1.5,4.3,5.015789,...,9.2,0.4,0.1,0.242105,0.2,0.5,5.0,0.2,38,5
3,4.6,3.1,1.5,0.2,setosa,0.111473,216.1,1.4,4.4,5.025581,...,10.6,0.5,0.1,0.246512,0.2,0.6,6.0,0.2,43,6
4,5.0,3.6,1.4,0.2,setosa,0.121577,185.7,1.5,4.3,5.018919,...,9.0,0.5,0.1,0.243243,0.2,0.6,6.0,0.2,37,6


In [11]:
len(X_train)

120

In [12]:
def target_encoding_test(X_train, X_test, key_col):
    agg = X_train.groupby(key_col).agg(aggs)
    names_1 = agg.columns.get_level_values(0)
    names_2 = agg.columns.get_level_values(1)
    agg.columns = ['agg_' + el1 + '_' + el2 for (el1, el2) in zip(names_1, names_2)]
    return pd.merge(X_test, agg, on=key_col, how='left')

In [13]:
X_test = target_encoding_test(X_train, X_test, 'species')

In [14]:
X_test.head()

,sepal_length,sepal_width,petal_length,petal_width,species,agg_sepal_length_var,agg_sepal_length_sum,agg_sepal_length_max_minus_min,agg_sepal_length_min,agg_sepal_length_mean,...,agg_agg_petal_width_nunique_sum,agg_agg_petal_width_nunique_max_minus_min,agg_agg_petal_width_nunique_min,agg_agg_petal_width_nunique_mean,agg_agg_petal_width_nunique_quantile,agg_agg_petal_width_nunique_max,agg_agg_petal_width_nunique_max_devided_min,agg_agg_petal_width_nunique_median,agg_agg_petal_width_nunique_count,agg_agg_petal_width_nunique_nunique
0,5.8,2.8,5.1,2.4,virginica,0.535158,131.2,2.8,4.9,6.560,...,185,1,9,9.25,9.0,10,1.111111,9,20,2
1,6.0,2.2,4.0,1.0,versicolor,0.266433,296.8,2.1,4.9,5.936,...,431,1,8,8.62,9.0,9,1.125000,9,50,2
2,5.5,4.2,1.4,0.2,setosa,0.124249,250.3,1.5,4.3,5.006,...,279,1,5,5.58,6.0,6,1.200000,6,50,2
3,7.3,2.9,6.3,1.8,virginica,0.535158,131.2,2.8,4.9,6.560,...,185,1,9,9.25,9.0,10,1.111111,9,20,2
4,5.0,3.4,1.5,0.2,setosa,0.124249,250.3,1.5,4.3,5.006,...,279,1,5,5.58,6.0,6,1.200000,6,50,2


In [15]:
len(X_test)

30

In [16]:
def get_skew_kurt_from_df(df, col_name_agg, by_hoge):
    gp = df.groupby(by_hoge)[col_name_agg].agg(['mean', 'std']).reset_index().rename(columns = {'mean' : 'mu', 'std' : 's'})
    merged = pd.merge(df, gp, how = 'left', on = by_hoge)
    merged['z'] = (merged[col_name_agg] - merged['mu'])/merged['s']
    merged['z_pow_3'] = merged['z'] ** 3
    merged['z_pow_4'] = merged['z'] ** 4
    gp_z_pow_sum = merged.groupby(by_hoge)[['z_pow_3', 'z_pow_4']].sum()
    gp = df.groupby(by_hoge).size().reset_index().rename(columns = {0 : 'n'})
    assert(gp.shape[0] == gp_z_pow_sum.shape[0])
    gp['z_pow_3_sum'] = gp_z_pow_sum['z_pow_3'].values
    gp['z_pow_4_sum'] = gp_z_pow_sum['z_pow_4'].values
    gp['skew'] = (gp['n']/((gp['n'] - 1) * (gp['n'] - 2))) * gp['z_pow_3_sum']
    gp['kurt'] = ((gp['n'] * (gp['n'] + 1))/((gp['n'] - 1) * (gp['n'] - 2) * (gp['n'] - 3))) * gp['z_pow_4_sum'] \
    - (3 * ((gp['n'] - 1) ** 2))/((gp['n'] - 2) * (gp['n'] - 3))
    gp = gp.drop(['n', 'z_pow_3_sum', 'z_pow_4_sum'], axis = 1)
    return gp

In [17]:
get_skew_kurt_from_df(df, 'sepal_length', 'species')

,species,skew,kurt
0,setosa,0.120087,-0.252689
1,versicolor,0.105378,-0.533010
2,virginica,0.118015,0.032904
